# Tools on LangChain

In [1]:
# Cell for loading my environment variables from config.py,
# which is located one directory above this notebook.
import sys
import os

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)

from config import set_environment
set_environment()

In [2]:
# from langchain_google_vertexai import ChatVertexAI
from langchain_openai import ChatOpenAI

# llm = ChatVertexAI(model="gemini-2.0-flash-001")
llm = ChatOpenAI()
question = "how old is the US president?"

Modern LLMs hide the need to construct a prompt from the user, you can define your tools as schemas instead and pass them as a separate argument:

In [3]:
search_tool = {
    "type": "function",
    "function": {
        "name": "google_search",
        "description": "Returns about common facts, fresh events and news from Google Search engine based on a query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "title": "search_query",
                    "description": "Search query to be sent to the search engine"
                }
            },
            "required": ["query"]
        }
    }
}
step1 = llm.invoke(question, tools=[search_tool])

As we can see, now our outputs contains a special part called `tool_calls` - again, there's no need for us to parse the output any more, it's all done by an LLM:

In [4]:
print(step1.tool_calls)

[{'name': 'google_search', 'args': {'query': 'current age of the US president'}, 'id': 'call_fNKQMstCvpceI7LNORZT4BLw', 'type': 'tool_call'}]


We can pass the tool calling result back to an LLM as a special `ToolMessage` message:

In [5]:
from langchain_core.messages import HumanMessage, ToolMessage

tool_result = ToolMessage(content="Donald Trump › Age 78 years June 14, 1946\n", tool_call_id=step1.tool_calls[0]["id"])
step2 = llm.invoke(
    [HumanMessage(content=question), step1, tool_result],
    tools=[search_tool]
)
assert len(step2.tool_calls) == 0

print(step2.content)

The current age of the US President, Donald Trump, is 78 years old. He was born on June 14, 1946.


For the convinience, we can also `bind` tools to an LLM so that they would be auto-added to arguments on every invocation:

In [6]:
llm_with_tools = llm.bind(tools=[search_tool])
llm_with_tools.invoke(question)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_E5TPwdGtjw6Y3RxcJoyuJHz7', 'function': {'arguments': '{"query":"current age of US president"}', 'name': 'google_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 80, 'total_tokens': 98, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BvPDtWCo9VxXcLqNkhFOvt05aMQzr', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--89d2f620-b048-4c7e-9a83-14d66ea93f26-0', tool_calls=[{'name': 'google_search', 'args': {'query': 'current age of US president'}, 'id': 'call_E5TPwdGtjw6Y3RxcJoyuJHz7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 80, 'output_tokens': 18, 'total_tokens': 